**Data Description**

The dataset used here is from Jones and Libicki 'How Terrorist Groups End' (Santa Monica, 2008, RAND Corporation). The full report can be found here:

http://www.rand.org/content/dam/rand/pubs/monographs/2008/RAND_MG741-1.pdf

- ***Authors***     - Jones and Libicki
- ***Data Nature*** - RAND - Oklahoma City National Memorial Institute for the Prevention of                           Terrorism (MIPT) Terrorism Incident database
- ***Purpose***     - To investigate the factors that contribute to end of terrorist groups. 

The data includes starting time for 648 terrorist groups that operated between 1968 and 2006.  

- 648 terrorist groups and 5 categorical variables. 

**Variables**

**Operating Peak Size**: The peak size of the group.

    Categories
        10,000s (10,000 or more)
        1,000s  (1,000-9,999)
        100s    (100-999)
        10s     (less than 100).

**Econ**: The income level of the base country, described according to World Bank criteria in             the year in which the group ended or in 2006 if the group has not ended.

    Categories
       H  = High income($10,726 or more)
       UM = upper middle income($3,466–$10,725)
       LM = lower middle income($876–$3,465)
       L  = low income($875 or less)

**Goal**: The primary goal of the terrorist group.

    Categories
        Regime_change
        Territorial_change
        Policy_change
        Empire
        Social_revolution
        Status_Quo
        
**Regime**: The level of freedom in the country against which the terrorist group operates.   

    Categories
        Not free
        Partly free
        Free values.

**Type**: The type of the terrorist group.
    
    Categories
        Left_wing
        Right_wing
        Nationalist
        

**Reason**: The reason for ending.
    
    Categories
        Policing
        Splintering
        Politics
        Victory
        Military_force
        Not ended        

**Time** and **Censored** variable to perform the survival analysis.

**Time** - Time in years since the start of the terrorist group's activities, either to the time when the terror group ended or to the last observation year (2006) if the terror group did not end until the end of the observed period (1968 - 2006).

**Censored** - The censoring variable is labeled 'Ended' and it equals to 1 if the terror group ended between 1968 and 2006 and it equals to 0 if the terror group did not end until 2006.

In [3]:
import pandas as pd
import os
from   PyPDF2               import PdfFileWriter, PdfFileReader
from   pdfminer.pdfinterp   import PDFResourceManager, PDFPageInterpreter
from   pdfminer.pdfpage     import PDFPage
from   pdfminer.converter   import XMLConverter, HTMLConverter, TextConverter
from   pdfminer.layout      import LAParams
from   pdfminer.pdfdocument import PDFDocument
from   pdfminer.pdfparser   import PDFParser
import io
import os
from   multiprocessing import Pool
import numpy as np
import pdfplumber
from   tabula import read_pdf

In [2]:
def split_pdf(file_name="../data/RAND_MG741-1.pdf"):
    
    inputpdf = PdfFileReader(open(file_name, "rb"))
    
    for i in range(inputpdf.numPages):
        output = PdfFileWriter()
        output.addPage(inputpdf.getPage(i))
        
        with open("document-page%s.pdf" % i, "wb") as outputStream:
            if outputStream:
                output.write(outputStream)

In [8]:
split_pdf()

In [18]:
# Data ranges from page number - 
# 168-211 

In [47]:
def single_create_text_data(page_number):
    
    file_name       = "document-page{}.pdf".format(page_number)
    df              = read_pdf(file_name)
    output_filename = "document-page{}.txt".format(page_number)
    df.to_csv(output_filename,sep='\t',encoding ='UTF-8')
    
    return   

In [48]:
def multiple_create_text_data(start_page=167,end_page=211):
    
    kfold           = 50
    mp_pool         = Pool(kfold)
    tasks           = np.arange(start_page,end_page)
    results_EDDM    = mp_pool.map(single_create_text_data, tasks)
    mp_pool.close()
    mp_pool.join()
    
    return 

In [49]:
multiple_create_text_data()

In [18]:
data = pd.read_csv("document-page168.csv",encoding ='UTF-8')

In [8]:
def data_massage(pageNumber):
    
    filename = "document-page{}.txt".format(pageNumber)
    data     = pd.read_csv(filename,encoding ='UTF-8')
    data.rename(columns={'Econ.a':'Econ','Organization Operating':'Organization','Regimeb':'Regime','Typec':'Type','Goald':'Goal','Endede':'Ended'},inplace=True)
    data.drop(['Unnamed: 0'],axis=1,inplace=True)
    n        = data.shape[0]
    
    if 'Unnamed: 1' in data.columns:
        data.drop(['Unnamed: 1'],axis=1,inplace=True)
    
    for i in range(n):
            try:
                if data.iloc[i].isna().sum() == 7:
                    data.drop([i],axis=0,inplace=True)
            except:
                pass
            
    return data

In [9]:
#tasks               = np.arange(168,211)
tasks               = np.arange(168,170)

In [10]:
def multi_data_massage(tasks):
    
    kfold           = 50
    mp_pool         = Pool(kfold)
    results_df      = mp_pool.map(data_massage, tasks)
    mp_pool.close()
    mp_pool.join()
    
    return results_df

In [11]:
df_list = multi_data_massage(tasks)

ParserError: Error tokenizing data. C error: Expected 2 fields in line 17, saw 3


In [28]:
df      = pd.DataFrame()
n       = len(df_list)

In [30]:
for i in range(n):
    #df = pd.concat([df,df_list[i][['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
    #     'Goal', 'Ended']]],axis=0)
    print(i,df_list[i].columns,'\n')

0 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

1 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

2 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

3 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

4 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

5 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

6 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='object') 

7 Index(['Organization', 'Operating', 'Peak Size', 'Econ', 'Regime', 'Type',
       'Goal', 'Ended'],
      dtype='obj